In [ ]:
pip install transformers torch datasets


In [ ]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments
from datasets import load_dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load the dataset (adjust the dataset loading according to your dataset)
import pandas as pd
trainning_df = pd.read_csv('/content/drive/MyDrive/train.csv')

In [ ]:
subset_df = trainning_df.iloc[0:5000,:].copy()

In [ ]:
pip install sentencepiece


In [ ]:
!pip show transformers
!pip show sentencepiece


Name: transformers
Version: 4.35.2
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: 
Name: sentencepiece
Version: 0.1.99
Summary: SentencePiece python wrapper
Home-page: https://github.com/google/sentencepiece
Author: Taku Kudo
Author-email: taku@google.com
License: Apache
Location: /usr/local/lib/python3.10/dist-packages
Requires: 
Required-by: 


In [ ]:
pip install accelerate -U

In [ ]:
pip install transformers[torch]

In [ ]:
from datasets import Dataset
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
import torch

dataset = Dataset.from_pandas(subset_df)

# Load the dataset from a CSV file
training_df = pd.read_csv('/content/drive/MyDrive/train.csv')
subset_df = training_df.iloc[0:5000, :].copy()

# Convert the DataFrame to a Hugging Face Dataset
dataset = Dataset.from_pandas(subset_df)

# Define the preprocessing function
def preprocess_function(examples):
    model_inputs = tokenizer(examples['article'], max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(examples['highlights'], max_length=128, truncation=True, padding="max_length")
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# Initialize T5 tokenizer
tokenizer = T5Tokenizer.from_pretrained('t5-small')

# Apply preprocessing - Now using the map method from the datasets library
tokenized_datasets = dataset.map(preprocess_function, batched=True)

# Split the dataset into training and validation sets if needed
train_dataset = tokenized_datasets.train_test_split(test_size=0.1)['train']
val_dataset = tokenized_datasets.train_test_split(test_size=0.1)['test']

# Load T5 model
model = T5ForConditionalGeneration.from_pretrained('t5-small')

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.resize_token_embeddings(len(tokenizer))
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    load_best_model_at_end=True,
    evaluation_strategy="steps"
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Train the model
trainer.train()

# Save the model and tokenizer after training
model.save_pretrained("/content/drive/MyDrive/")
tokenizer.save_pretrained("/content/drive/MyDrive/")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Step,Training Loss,Validation Loss
500,3.804600,1.131254
1000,1.216600,1.086312
1500,1.178600,1.069685
2000,1.153600,1.057054
2500,1.151200,1.051152
3000,1.131100,1.046748


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


('/content/drive/MyDrive/tokenizer_config.json',
 '/content/drive/MyDrive/special_tokens_map.json',
 '/content/drive/MyDrive/spiece.model',
 '/content/drive/MyDrive/added_tokens.json')

In [ ]:
input_text = "That spider you squished? It could have been used for science! At least, that's what Faye Yap and Daniel Preston think. Yap is a mechanical engineering PhD student in Preston's lab at Rice University, where she co-authored a paper on reanimating spider corpses to create grippers, or tiny machines used to pick up and put down delicate objects. Yap and Preston dubbed this use of biotic materials for robotic parts necrobotics – and think this technique could one day become a cheap, green addition to the field."
input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True, padding="max_length").to(device)


In [ ]:
with torch.no_grad():
    output = model.generate(input_ids, max_length=150, num_return_sequences=1, early_stopping=True)

generated_summary = tokenizer.decode(output[0], skip_special_tokens=True)


In [ ]:
print("Generated Summary:")
print(generated_summary)


Generated Summary:
Faye Yap and Daniel Preston think this technique could have been used for science. They think it could have been used for science. Yap and Preston dubbed this use of biotic materials for robotic parts necrobotics.
